In [7]:
import cv2
import numpy as np
import requests 
import os.path  

def download_image(url, filename):

    if os.path.exists(filename):
        print(f"File '{filename}' already exists. Skipping download.")
        return True
    
    print(f"Downloading '{filename}' from {url}...")
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status() 
        
        with open(filename, 'wb') as f:
            f.write(response.content)
            
        print(f"Successfully downloaded '{filename}'.")
        return True
    except requests.exceptions.RequestException as e:
        print(f"Error: Could not download image. {e}")
        return False


def stitch_images(img_left, img_right, min_match_count=10):

    gray_left = cv2.cvtColor(img_left, cv2.COLOR_BGR2GRAY)
    gray_right = cv2.cvtColor(img_right, cv2.COLOR_BGR2GRAY)

    try:
        sift = cv2.SIFT_create()
        kp_left, des_left = sift.detectAndCompute(gray_left, None)
        kp_right, des_right = sift.detectAndCompute(gray_right, None)
    except cv2.error as e:
        print("\n--- ERROR ---")
        print("OpenCV Error. This often means your opencv-contrib-python package is missing or not installed correctly.")
        print("Please run: pip install opencv-contrib-python")
        print("-------------\n")
        raise e

    FLANN_INDEX_KDTREE = 1
    index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
    search_params = dict(checks=50)
    
    flann = cv2.FlannBasedMatcher(index_params, search_params)
    matches = flann.knnMatch(des_right, des_left, k=2)

    good_matches = []
    for m, n in matches:
        if m.distance < 0.75 * n.distance:
            good_matches.append(m)

    if len(good_matches) < min_match_count:
        print(f"Not enough good matches found - {len(good_matches)}/{min_match_count}")
        return None

    src_pts = np.float32([kp_right[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
    dst_pts = np.float32([kp_left[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)

    M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)

    h_left, w_left = img_left.shape[:2]
    h_right, w_right = img_right.shape[:2]
    warped_right = cv2.warpPerspective(img_right, M, (w_left + w_right, h_left))

    unblended_output = warped_right.copy()
    unblended_output[0:h_left, 0:w_left] = img_left

    aligned_left = np.zeros((h_left, w_left + w_right, 3), dtype=np.uint8)
    aligned_left[0:h_left, 0:w_left] = img_left
    mask_R = np.zeros((h_left, w_left + w_right), dtype=np.float32)
    mask_R[0:h_left, 0:w_left] = 1.0
    
    print("2. Stitching complete. Ready for blending.")
    return aligned_left, warped_right, mask_R, unblended_output


def laplacian_blend(A, B, R, levels=6):

    def build_gaussian_pyramid(img, levels):
        G = [img]
        for i in range(levels):
            img = cv2.pyrDown(img)
            G.append(img)
        return G

    def build_laplacian_pyramid(img, levels):
        G = build_gaussian_pyramid(img, levels)
        L = []
        for i in range(levels):
            h, w = G[i].shape[:2]
            upsampled = cv2.pyrUp(G[i+1], dstsize=(w, h))
            L.append(cv2.subtract(G[i], upsampled))
        L.append(G[levels])
        return L

    def collapse_pyramid(L):
        img = L[-1]
        for i in range(len(L)-2, -1, -1):
            h, w = L[i].shape[:2]
            upsampled = cv2.pyrUp(img, dstsize=(w, h))
            img = cv2.add(upsampled, L[i])
        return img

    print("3. Starting Laplacian Blending...")

    LA = build_laplacian_pyramid(A, levels)
    LB = build_laplacian_pyramid(B, levels)

    GR = build_gaussian_pyramid(R, levels)

    LS = []
    for la, lb, gr in zip(LA, LB, GR):
        gr_expanded = cv2.merge([gr, gr, gr])
        ls = gr_expanded * la + (1.0 - gr_expanded) * lb
        LS.append(ls)

    blended_image = collapse_pyramid(LS)
    
    print("4. Blending complete.")
    return blended_image

if __name__ == "__main__":

    IMG_LEFT_PATH = "pano_left.jpg"
    IMG_RIGHT_PATH = "pano_right.jpg"
    URL_LEFT = "https://raw.githubusercontent.com/tranleanh/image-panorama-stitching/master/images/left.jpg"
    URL_RIGHT = "https://raw.githubusercontent.com/tranleanh/image-panorama-stitching/master/images/right.jpg"

    if not (download_image(URL_LEFT, IMG_LEFT_PATH) and download_image(URL_RIGHT, IMG_RIGHT_PATH)):
        print("Could not download one or more images. Exiting.")
        exit()

    img_left = cv2.imread(IMG_LEFT_PATH)
    img_right = cv2.imread(IMG_RIGHT_PATH)

    if img_left is None or img_right is None:
        print(f"Error: Could not load images even after download. Check file permissions.")
        exit()

    try:
        stitch_result = stitch_images(img_left, img_right)
    except Exception as e:
        print(f"An error occurred during stitching: {e}")
        exit()
        
    if stitch_result is None:
        print("Stitching failed. Exiting.")
        exit()

    aligned_left, aligned_right, mask_R, unblended = stitch_result

    A = aligned_left.astype(np.float32) / 255.0
    B = aligned_right.astype(np.float32) / 255.0

    blended = laplacian_blend(A, B, mask_R, levels=6)

    blended_final = (np.clip(blended, 0, 1) * 255).astype(np.uint8)

    cv2.imwrite("unblended_panorama.jpg", unblended)
    cv2.imwrite("blended_panorama.jpg", blended_final)

    print("\nDone. Successfully saved:")
    print("- unblended_panorama.jpg (the stitched image with the seam)")
    print("- blended_panorama.jpg (the final blended image)")

File 'pano_left.jpg' already exists. Skipping download.
File 'pano_right.jpg' already exists. Skipping download.
2. Stitching complete. Ready for blending.
3. Starting Laplacian Blending...
4. Blending complete.

Done. Successfully saved:
- unblended_panorama.jpg (the stitched image with the seam)
- blended_panorama.jpg (the final blended image)
